In [1]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
# from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
# from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from datetime import datetime
import time
import math

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [346]:
train_X = pd.read_csv('./data/train.csv')
test_X = pd.read_csv('./data/test.csv')
#submission = pd.read_csv('./sample_submission.csv')

# 밀린 칼럼 옮기기

In [347]:
# 조건에 맞는 행 필터링
condition = (train_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] == 'OK') | (train_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'].isna())
condition_indices = train_X[condition].index

# 열 범위 설정
start_col = train_X.columns.get_loc('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam')
end_col = train_X.columns.get_loc('WorkMode Collect Result_Dam') + 1
cols_to_shift = train_X.columns[start_col:end_col]

# 조건에 맞는 행의 특정 열 이후의 값을 왼쪽으로 3칸 이동
train_X.loc[condition_indices, cols_to_shift] = train_X.loc[condition_indices, cols_to_shift].apply(lambda row: row.shift(-3), axis=1)

# 결과 확인
train_X


,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [348]:
# 조건에 맞는 행 필터링
condition = (test_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] == 'OK') | (test_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'].isna())
condition_indices = test_X[condition].index

# 열 범위 설정
start_col = test_X.columns.get_loc('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam')
end_col = test_X.columns.get_loc('WorkMode Collect Result_Dam') + 1
cols_to_shift = test_X.columns[start_col:end_col]

# 조건에 맞는 행의 특정 열 이후의 값을 왼쪽으로 3칸 이동
test_X.loc[condition_indices, cols_to_shift] = test_X.loc[condition_indices, cols_to_shift].apply(lambda row: row.shift(-3), axis=1)

# 결과 확인
test_X

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000.0,NaN,...,195,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000.0,NaN,...,14,NaN,NaN,256,NaN,NaN,1,NaN,NaN,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240.0,NaN,...,98,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000.0,NaN,...,14,NaN,NaN,0,NaN,NaN,1,NaN,NaN,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240.0,NaN,...,1,NaN,NaN,215,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,NaN,...,14,NaN,NaN,131,NaN,NaN,1,NaN,NaN,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,NaN,...,12,NaN,NaN,279,NaN,NaN,1,NaN,NaN,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,NaN,...,4,NaN,NaN,66,NaN,NaN,1,NaN,NaN,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN


In [349]:
# 조건에 맞는 행 필터링
condition = (train_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] == 'OK') | (pd.isna(train_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))
condition_indices = train_X[condition].index

# 'Collect Result.17' 열 이후의 값을 왼쪽으로 이동
start_col = train_X.columns.get_loc('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1')
end_col = train_X.columns.get_loc('WorkMode Collect Result_Fill1') + 1

cols_to_shift = train_X.columns[start_col:end_col]


train_X.loc[condition_indices, cols_to_shift] = train_X.loc[condition_indices, cols_to_shift].apply(lambda row: row.shift(-3), axis=1)

# 결과 확인
train_X


,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [350]:
# 조건에 맞는 행 필터링
condition = (test_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] == 'OK') | (pd.isna(test_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))
condition_indices = test_X[condition].index

# 'Collect Result.17' 열 이후의 값을 왼쪽으로 이동
start_col = test_X.columns.get_loc('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1')
end_col = test_X.columns.get_loc('WorkMode Collect Result_Fill1') + 1

cols_to_shift = test_X.columns[start_col:end_col]


test_X.loc[condition_indices, cols_to_shift] = test_X.loc[condition_indices, cols_to_shift].apply(lambda row: row.shift(-3), axis=1)

# 결과 확인
test_X


,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000.0,NaN,...,195,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000.0,NaN,...,14,NaN,NaN,256,NaN,NaN,1,NaN,NaN,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240.0,NaN,...,98,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000.0,NaN,...,14,NaN,NaN,0,NaN,NaN,1,NaN,NaN,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240.0,NaN,...,1,NaN,NaN,215,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,NaN,...,14,NaN,NaN,131,NaN,NaN,1,NaN,NaN,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,NaN,...,12,NaN,NaN,279,NaN,NaN,1,NaN,NaN,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,NaN,...,4,NaN,NaN,66,NaN,NaN,1,NaN,NaN,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN


In [351]:
# 조건에 맞는 행 필터링
condition = (train_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] == 'OK') | (pd.isna(train_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))
condition_indices = train_X[condition].index

# 'Collect Result.17' 열 이후의 값을 왼쪽으로 이동
start_col = train_X.columns.get_loc('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2')
end_col = train_X.columns.get_loc('WorkMode Collect Result_Fill2') + 1

cols_to_shift = train_X.columns[start_col:end_col]


train_X.loc[condition_indices, cols_to_shift] = train_X.loc[condition_indices, cols_to_shift].apply(lambda row: row.shift(-3), axis=1)

# 결과 확인
train_X


,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,127,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,73,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,197,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,27,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal


In [352]:
# 조건에 맞는 행 필터링
condition = (test_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] == 'OK') | (pd.isna(test_X['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))
condition_indices = test_X[condition].index

# 'Collect Result.17' 열 이후의 값을 왼쪽으로 이동
start_col = test_X.columns.get_loc('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2')
end_col = test_X.columns.get_loc('WorkMode Collect Result_Fill2') + 1

cols_to_shift = test_X.columns[start_col:end_col]


test_X.loc[condition_indices, cols_to_shift] = test_X.loc[condition_indices, cols_to_shift].apply(lambda row: row.shift(-3), axis=1)

# 결과 확인
test_X


,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000.0,NaN,...,195,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000.0,NaN,...,256,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240.0,NaN,...,98,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000.0,NaN,...,0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240.0,NaN,...,215,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,NaN,...,131,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,NaN,...,279,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,NaN,...,66,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,NaN,...,117,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,NaN


In [353]:
all_data = pd.concat([train_X, test_X], sort=False)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57867 entries, 0 to 17360
Columns: 465 entries, Wip Line_Dam to Set ID
dtypes: float64(359), int64(68), object(38)
memory usage: 205.7+ MB


In [354]:
all_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target,Set ID
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal,NaN
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal,NaN
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal,NaN
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal,NaN
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,ffea508b59934d689b540f95eb3fa730
17357,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,ffed8923c8a448a98afc641b770be153
17358,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,fff1e73734da40adbe805359b3efb462
17359,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,NaN,fff8e38bdd09470baf95f71e92075dec


# 값이 1개도 없거나 1개뿐인 열 제거하기

In [355]:
all_data['CURE DpS X Collect Result_Dam'] = (abs(all_data['CURE STANDBY POSITION X Collect Result_Dam']-all_data['CURE START POSITION X Collect Result_Dam']) + abs(all_data['CURE START POSITION X Collect Result_Dam']-all_data['CURE END POSITION X Collect Result_Dam']))/all_data['CURE SPEED Collect Result_Dam']
all_data['CURE DpS Z Collect Result_Dam'] = (abs(all_data['CURE STANDBY POSITION Z Collect Result_Dam']-all_data['CURE START POSITION Z Collect Result_Dam']) + abs(all_data['CURE START POSITION Z Collect Result_Dam']-all_data['CURE END POSITION Z Collect Result_Dam']))/all_data['CURE SPEED Collect Result_Dam']
all_data['CURE DpS Θ Collect Result_Dam'] = (abs(all_data['CURE STANDBY POSITION Θ Collect Result_Dam']-all_data['CURE START POSITION Θ Collect Result_Dam']) + abs(all_data['CURE START POSITION Θ Collect Result_Dam']-all_data['CURE END POSITION Θ Collect Result_Dam']))/all_data['CURE SPEED Collect Result_Dam']

all_data['CURE DpS X Collect Result_Fill2'] = (abs(all_data['CURE STANDBY POSITION X Collect Result_Fill2']-all_data['CURE START POSITION X Collect Result_Fill2']) + abs(all_data['CURE START POSITION X Collect Result_Fill2']-all_data['CURE END POSITION X Collect Result_Fill2']))/all_data['CURE SPEED Collect Result_Fill2']
all_data['CURE DpS Z Collect Result_Fill2'] = (abs(all_data['CURE STANDBY POSITION Z Collect Result_Fill2']-all_data['CURE START POSITION Z Collect Result_Fill2']) + abs(all_data['CURE START POSITION Z Collect Result_Fill2']-all_data['CURE END POSITION Z Collect Result_Fill2']))/all_data['CURE SPEED Collect Result_Fill2']
all_data['CURE DpS Θ Collect Result_Fill2'] = (abs(all_data['CURE STANDBY POSITION Θ Collect Result_Fill2']-all_data['CURE START POSITION Θ Collect Result_Fill2']) + abs(all_data['CURE START POSITION Θ Collect Result_Fill2']-all_data['CURE END POSITION Θ Collect Result_Fill2']))/all_data['CURE SPEED Collect Result_Fill2']


# 기존 열 삭제
all_data = all_data.drop(columns=['CURE STANDBY POSITION X Collect Result_Dam','CURE STANDBY POSITION Z Collect Result_Dam','CURE STANDBY POSITION Θ Collect Result_Dam',
                                  'CURE START POSITION X Collect Result_Dam','CURE START POSITION Z Collect Result_Dam','CURE START POSITION Θ Collect Result_Dam',
                                  'CURE END POSITION X Collect Result_Dam','CURE END POSITION Z Collect Result_Dam','CURE END POSITION Θ Collect Result_Dam','CURE SPEED Collect Result_Dam',
                                  'CURE STANDBY POSITION X Collect Result_Fill2','CURE STANDBY POSITION Z Collect Result_Fill2','CURE STANDBY POSITION Θ Collect Result_Fill2',
                                  'CURE START POSITION X Collect Result_Fill2','CURE START POSITION Z Collect Result_Fill2','CURE START POSITION Θ Collect Result_Fill2',
                                  'CURE END POSITION X Collect Result_Fill2','CURE END POSITION Z Collect Result_Fill2','CURE END POSITION Θ Collect Result_Fill2','CURE SPEED Collect Result_Fill2'])

# 결과 확인
all_data.head()


,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,CURE END POSITION Z Unit Time_Dam,...,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target,Set ID,CURE DpS X Collect Result_Dam,CURE DpS Z Collect Result_Dam,CURE DpS Θ Collect Result_Dam,CURE DpS X Collect Result_Fill2,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,NaN,NaN,NaN,...,NaN,NaN,Normal,NaN,9.100000,0.310000,0.900000,16.25,0.00,5.625
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,NaN,NaN,NaN,...,NaN,NaN,Normal,NaN,13.000000,0.442857,1.285714,15.60,0.00,5.400
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,NaN,NaN,NaN,...,NaN,NaN,Normal,NaN,18.705882,0.247059,1.058824,15.60,0.00,5.400
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,NaN,NaN,NaN,...,NaN,NaN,Normal,NaN,22.714286,0.300000,1.285714,15.60,0.00,5.400
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,NaN,NaN,NaN,...,NaN,NaN,Normal,NaN,13.000000,0.442857,1.285714,15.60,0.02,5.400


In [356]:
cols = ['Dispense Volume(Stage1) Collect Result_Dam','Dispense Volume(Stage2) Collect Result_Dam','Dispense Volume(Stage3) Collect Result_Dam',
        'Dispense Volume(Stage1) Collect Result_Fill1','Dispense Volume(Stage2) Collect Result_Fill1','Dispense Volume(Stage3) Collect Result_Fill1',
        'Dispense Volume(Stage1) Collect Result_Fill2','Dispense Volume(Stage2) Collect Result_Fill2','Dispense Volume(Stage3) Collect Result_Fill2',
        'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam','DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam','DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
        'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1','DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1','DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1',
        'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2','DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2','DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2'
        ]


a = all_data[cols[0]] / all_data[cols[9]]
b = all_data[cols[1]] / all_data[cols[10]]
c = all_data[cols[2]] / all_data[cols[11]]

all_data['DISCHARGED VpS RESIN(Stage) Collect Result_Dam'] =(a+b+c)/3

a = all_data[cols[4]] / all_data[cols[12]]
b = all_data[cols[5]] / all_data[cols[13]]
c = all_data[cols[6]] / all_data[cols[14]]

all_data['DISCHARGED VpS RESIN(Stage) Collect Result_Dam'] =(a+b+c)/3

a = all_data[cols[7]] / all_data[cols[15]]
b = all_data[cols[8]] / all_data[cols[16]]
c = all_data[cols[9]] / all_data[cols[17]]

all_data['DISCHARGED VpS RESIN(Stage) Collect Result_Dam'] =(a+b+c)/3

all_data = all_data.drop(columns=cols)

# 결과 확인
all_data.head()

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,CURE END POSITION Z Unit Time_Dam,...,WorkMode Judge Value_Fill2,target,Set ID,CURE DpS X Collect Result_Dam,CURE DpS Z Collect Result_Dam,CURE DpS Θ Collect Result_Dam,CURE DpS X Collect Result_Fill2,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2,DISCHARGED VpS RESIN(Stage) Collect Result_Dam
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,NaN,NaN,NaN,...,NaN,Normal,NaN,9.100000,0.310000,0.900000,16.25,0.00,5.625,NaN
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,NaN,NaN,NaN,...,NaN,Normal,NaN,13.000000,0.442857,1.285714,15.60,0.00,5.400,NaN
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,NaN,NaN,NaN,...,NaN,Normal,NaN,18.705882,0.247059,1.058824,15.60,0.00,5.400,NaN
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,NaN,NaN,NaN,...,NaN,Normal,NaN,22.714286,0.300000,1.285714,15.60,0.00,5.400,NaN
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,NaN,NaN,NaN,...,NaN,Normal,NaN,13.000000,0.442857,1.285714,15.60,0.02,5.400,NaN


In [357]:
# a = []
# A = pd.DataFrame(train_X.isnull().sum(),columns=['sum'])
# a = list(A[A['sum']==train_X.shape[0]].index)
# train_X.drop(columns=a, inplace=True)


# a=[]
# for c in train_X.columns:
#     if train_X[c].nunique()==1:
#         a.append(c)
# train_X.drop(columns=a, inplace=True)

# a = []
# A = pd.DataFrame(test_X.isnull().sum(),columns=['sum'])
# a = list(A[A['sum']==test_X.shape[0]].index)
# test_X.drop(columns=a, inplace=True)


# a=[]
# for c in test_X.columns:
#     if test_X[c].nunique()==1:
#         a.append(c)
# test_X.drop(columns=a, inplace=True)

a = []
A = pd.DataFrame(all_data.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==all_data.shape[0]].index)
all_data.drop(columns=a, inplace=True)


a=[]
for c in all_data.columns:
    if all_data[c].nunique()==1:
        a.append(c)
all_data.drop(columns=a, inplace=True)

In [358]:
all_data

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,...,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,target,Set ID,CURE DpS X Collect Result_Dam,CURE DpS Z Collect Result_Dam,CURE DpS Θ Collect Result_Dam,CURE DpS X Collect Result_Fill2,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2
0,Dam dispenser #1,AJX75334505,4F1XA938-1,16,550.5,464.3,161.4,384.0,383.1,384.0,...,127,1,Normal,NaN,9.100000,0.310000,0.900000,16.250000,0.000000,5.625000
1,Dam dispenser #1,AJX75334505,3KPM0016-2,10,550.3,463.8,160.8,377.3,377.3,377.3,...,185,1,Normal,NaN,13.000000,0.442857,1.285714,15.600000,0.000000,5.400000
2,Dam dispenser #2,AJX75334501,4E1X9167-1,16,162.4,465.4,551.7,1271.8,1383.9,1271.8,...,73,1,Normal,NaN,18.705882,0.247059,1.058824,15.600000,0.000000,5.400000
3,Dam dispenser #2,AJX75334501,3K1X0057-1,10,162.4,465.5,552.0,1271.8,1270.7,1271.8,...,268,1,Normal,NaN,22.714286,0.300000,1.285714,15.600000,0.000000,5.400000
4,Dam dispenser #1,AJX75334501,3HPM0007-1,10,549,462.5,159.5,377.5,377.0,377.5,...,121,1,Normal,NaN,13.000000,0.442857,1.285714,15.600000,0.020000,5.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,10,162.4,465.5,552.0,1271.8,1270.7,1271.8,...,131,1,NaN,ffea508b59934d689b540f95eb3fa730,22.714286,0.300000,1.285714,15.600000,0.000000,5.400000
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,16,162.7,465.7,552.0,1271.8,1383.9,1271.8,...,279,1,NaN,ffed8923c8a448a98afc641b770be153,22.714286,0.300000,1.285714,15.600000,0.000000,5.400000
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,16,551.5,465.0,162.3,377.6,377.1,377.6,...,66,1,NaN,fff1e73734da40adbe805359b3efb462,13.000000,0.442857,1.285714,15.600000,0.000000,5.400000
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,10,549,463.0,160.5,377.5,377.0,377.5,...,117,1,NaN,fff8e38bdd09470baf95f71e92075dec,13.000000,0.442857,1.285714,15.294118,0.196078,5.294118


In [359]:
all_data.describe(include='all')

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,...,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,target,Set ID,CURE DpS X Collect Result_Dam,CURE DpS Z Collect Result_Dam,CURE DpS Θ Collect Result_Dam,CURE DpS X Collect Result_Fill2,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2
count,57867,57867,57867,57867.000000,57867,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000,...,57867.000000,57867.000000,40506,17361,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000
unique,2,7,664,NaN,38,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,17361,NaN,NaN,NaN,NaN,NaN,NaN
top,Dam dispenser #1,AJX75334501,3L1X8574-2,NaN,549,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Normal,d1b6c8b0a3bb460e8ae4d7d1c82c818f,NaN,NaN,NaN,NaN,NaN,NaN
freq,35764,48315,380,NaN,10422,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,38156,1,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,12.890352,NaN,464.443052,310.354465,719.387470,741.666473,719.426307,...,176.992863,1.175834,NaN,NaN,16.009665,0.373360,1.234538,15.781758,0.004755,5.444933
std,NaN,NaN,NaN,2.997935,NaN,1.344287,190.084174,434.547683,464.784847,434.527894,...,118.772472,1.114499,NaN,NaN,4.730663,0.078786,0.125731,1.052496,0.023837,0.159448
min,NaN,NaN,NaN,10.000000,NaN,462.200000,159.500000,377.300000,377.000000,377.300000,...,0.000000,1.000000,NaN,NaN,8.666667,0.200000,0.857143,10.400000,0.000000,3.600000
25%,NaN,NaN,NaN,10.000000,NaN,463.500000,160.500000,377.500000,377.100000,377.500000,...,78.000000,1.000000,NaN,NaN,13.000000,0.300000,1.285714,15.600000,0.000000,5.400000
50%,NaN,NaN,NaN,10.000000,NaN,464.200000,161.500000,377.600000,377.300000,377.600000,...,163.000000,1.000000,NaN,NaN,13.000000,0.442857,1.285714,15.600000,0.000000,5.400000
75%,NaN,NaN,NaN,16.000000,NaN,465.400000,551.700000,1271.800000,1270.700000,1271.800000,...,259.000000,1.000000,NaN,NaN,22.714286,0.442857,1.285714,15.600000,0.000000,5.400000


# 중복되는 값 제거

In [360]:
l1=['Model.Suffix_AutoClave','Model.Suffix_Fill1','Model.Suffix_Fill2','Workorder_AutoClave','Workorder_Fill1','Workorder_Fill2','Receip No Collect Result_Fill1','Receip No Collect Result_Fill2']
all_data = all_data.drop(l1, axis=1)

In [361]:
all_data.head()

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,...,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,target,Set ID,CURE DpS X Collect Result_Dam,CURE DpS Z Collect Result_Dam,CURE DpS Θ Collect Result_Dam,CURE DpS X Collect Result_Fill2,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2
0,Dam dispenser #1,AJX75334505,4F1XA938-1,16,550.5,464.3,161.4,384.0,383.1,384.0,...,7.0,127,Normal,NaN,9.100000,0.310000,0.900000,16.25,0.00,5.625
1,Dam dispenser #1,AJX75334505,3KPM0016-2,10,550.3,463.8,160.8,377.3,377.3,377.3,...,7.0,185,Normal,NaN,13.000000,0.442857,1.285714,15.60,0.00,5.400
2,Dam dispenser #2,AJX75334501,4E1X9167-1,16,162.4,465.4,551.7,1271.8,1383.9,1271.8,...,10.0,73,Normal,NaN,18.705882,0.247059,1.058824,15.60,0.00,5.400
3,Dam dispenser #2,AJX75334501,3K1X0057-1,10,162.4,465.5,552.0,1271.8,1270.7,1271.8,...,12.0,268,Normal,NaN,22.714286,0.300000,1.285714,15.60,0.00,5.400
4,Dam dispenser #1,AJX75334501,3HPM0007-1,10,549,462.5,159.5,377.5,377.0,377.5,...,8.0,121,Normal,NaN,13.000000,0.442857,1.285714,15.60,0.02,5.400


# 중복되는 값 평균으로 합치기

In [362]:
# 열 목록 정의
cols_to_average = ['Stage1 Circle1 Distance Speed Collect Result_Dam',
    'Stage1 Circle2 Distance Speed Collect Result_Dam',
    'Stage1 Circle3 Distance Speed Collect Result_Dam',
    'Stage1 Circle4 Distance Speed Collect Result_Dam',
    'Stage1 Line1 Distance Speed Collect Result_Dam',
    'Stage1 Line2 Distance Speed Collect Result_Dam',
    'Stage1 Line3 Distance Speed Collect Result_Dam',
    'Stage1 Line4 Distance Speed Collect Result_Dam',
    'Stage2 Circle1 Distance Speed Collect Result_Dam',
    'Stage2 Circle2 Distance Speed Collect Result_Dam',
    'Stage2 Circle3 Distance Speed Collect Result_Dam',
    'Stage2 Circle4 Distance Speed Collect Result_Dam',
    'Stage2 Line1 Distance Speed Collect Result_Dam',
    'Stage2 Line2 Distance Speed Collect Result_Dam',
    'Stage2 Line3 Distance Speed Collect Result_Dam',
    'Stage2 Line4 Distance Speed Collect Result_Dam',
    'Stage3 Circle1 Distance Speed Collect Result_Dam',
    'Stage3 Circle2 Distance Speed Collect Result_Dam',
    'Stage3 Circle3 Distance Speed Collect Result_Dam',
    'Stage3 Circle4 Distance Speed Collect Result_Dam',
    'Stage3 Line1 Distance Speed Collect Result_Dam',
    'Stage3 Line2 Distance Speed Collect Result_Dam',
    'Stage3 Line3 Distance Speed Collect Result_Dam',
    'Stage3 Line4 Distance Speed Collect Result_Dam'
]

# 평균 값 계산 및 새로운 열 추가
# all_data['Average Stage1 Circle Distance Speed Collect Result_Dam'] = all_data[cols_to_average[:4]].mean(axis=1)
# all_data['Average Stage1 Line Distance Speed Collect Result_Dam'] = all_data[cols_to_average[4:8]].mean(axis=1)
# all_data['Average Stage2 Circle Distance Speed Collect Result_Dam'] = all_data[cols_to_average[8:12]].mean(axis=1)
# all_data['Average Stage2 Line Distance Speed Collect Result_Dam'] = all_data[cols_to_average[12:16]].mean(axis=1)
# all_data['Average Stage3 Circle Distance Speed Collect Result_Dam'] = all_data[cols_to_average[16:20]].mean(axis=1)
# all_data['Average Stage3 Line Distance Speed Collect Result_Dam'] = all_data[cols_to_average[20:24]].mean(axis=1)

all_data['Average Stage1 CL Distance Speed Collect Result_Dam'] = all_data[cols_to_average[:8]].mean(axis=1)
all_data['Average Stage2 CL Distance Speed Collect Result_Dam'] = all_data[cols_to_average[8:16]].mean(axis=1)
all_data['Average Stage3 CL Distance Speed Collect Result_Dam'] = all_data[cols_to_average[16:]].mean(axis=1)

# 기존 열 삭제
all_data = all_data.drop(columns=cols_to_average)

# 결과 확인
all_data.head()


,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,...,Set ID,CURE DpS X Collect Result_Dam,CURE DpS Z Collect Result_Dam,CURE DpS Θ Collect Result_Dam,CURE DpS X Collect Result_Fill2,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2,Average Stage1 CL Distance Speed Collect Result_Dam,Average Stage2 CL Distance Speed Collect Result_Dam,Average Stage3 CL Distance Speed Collect Result_Dam
0,Dam dispenser #1,AJX75334505,4F1XA938-1,16,550.5,464.3,161.4,384.0,383.1,384.0,...,NaN,9.100000,0.310000,0.900000,16.25,0.00,5.625,5775.0,5300.0,5800.0
1,Dam dispenser #1,AJX75334505,3KPM0016-2,10,550.3,463.8,160.8,377.3,377.3,377.3,...,NaN,13.000000,0.442857,1.285714,15.60,0.00,5.400,4000.0,9000.0,4000.0
2,Dam dispenser #2,AJX75334501,4E1X9167-1,16,162.4,465.4,551.7,1271.8,1383.9,1271.8,...,NaN,18.705882,0.247059,1.058824,15.60,0.00,5.400,5800.0,5300.0,5800.0
3,Dam dispenser #2,AJX75334501,3K1X0057-1,10,162.4,465.5,552.0,1271.8,1270.7,1271.8,...,NaN,22.714286,0.300000,1.285714,15.60,0.00,5.400,4000.0,5000.0,4000.0
4,Dam dispenser #1,AJX75334501,3HPM0007-1,10,549,462.5,159.5,377.5,377.0,377.5,...,NaN,13.000000,0.442857,1.285714,15.60,0.02,5.400,9000.0,9000.0,9000.0


In [363]:
cols_to_average = ['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
                   'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
                   'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
                   'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam',
                   'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
                    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam',
                    'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
                   'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
                   'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
                   'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
                   'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
                   'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2',
                   'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1',
                   'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1',
                    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1',
                   'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2',
                   'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2',
                    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'
]

all_data['HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam'] = all_data[cols_to_average[:3]].mean(axis=1)
all_data['HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Dam'] = all_data[cols_to_average[3:6]].mean(axis=1)

all_data['HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill'] = all_data[cols_to_average[6:12]].mean(axis=1)
all_data['HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill'] = all_data[cols_to_average[12:18]].mean(axis=1)




# 기존 열 삭제
all_data = all_data.drop(columns=cols_to_average)

# 결과 확인
all_data.head()



,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,...,CURE DpS X Collect Result_Fill2,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2,Average Stage1 CL Distance Speed Collect Result_Dam,Average Stage2 CL Distance Speed Collect Result_Dam,Average Stage3 CL Distance Speed Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill
0,Dam dispenser #1,AJX75334505,4F1XA938-1,16,550.5,464.3,161.4,124.00,130.85,300.0,...,16.25,0.00,5.625,5775.0,5300.0,5800.0,383.700000,274.51,429.000000,244.1100
1,Dam dispenser #1,AJX75334505,3KPM0016-2,10,550.3,463.8,160.8,130.85,130.85,300.0,...,15.60,0.00,5.400,4000.0,9000.0,4000.0,377.300000,282.15,429.283333,244.0500
2,Dam dispenser #2,AJX75334501,4E1X9167-1,16,162.4,465.4,551.7,124.00,130.85,300.0,...,15.60,0.00,5.400,5800.0,5300.0,5800.0,1309.166667,274.33,1323.616667,243.9575
3,Dam dispenser #2,AJX75334501,3K1X0057-1,10,162.4,465.5,552.0,130.85,130.85,300.0,...,15.60,0.00,5.400,4000.0,5000.0,4000.0,1271.433333,282.15,1323.516667,243.9000
4,Dam dispenser #1,AJX75334501,3HPM0007-1,10,549,462.5,159.5,133.50,133.50,300.0,...,15.60,0.02,5.400,9000.0,9000.0,9000.0,377.333333,284.80,429.183333,244.0500


In [364]:
# 열 목록 정의
columns_to_convert = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 각 열을 float 타입으로 변환
for col in columns_to_convert:
    all_data[col] = all_data[col].astype(float)

# 결과 확인
all_data[columns_to_convert].dtypes


HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam      float64
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1    float64
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2    float64
dtype: object

In [365]:
cols = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
        'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
        'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
        
        'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
        'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
        'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',]

all_data['HEAD NORMAL COORDINATE X AXIS(Stage) Collect Result_Fill'] = all_data[cols].sum(axis=1)/2

# 기존 열 삭제
all_data = all_data.drop(columns= cols)

# 결과 확인
all_data.head()


,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,...,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2,Average Stage1 CL Distance Speed Collect Result_Dam,Average Stage2 CL Distance Speed Collect Result_Dam,Average Stage3 CL Distance Speed Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE X AXIS(Stage) Collect Result_Fill
0,Dam dispenser #1,AJX75334505,4F1XA938-1,16,550.5,464.3,161.4,124.00,130.85,300.0,...,0.00,5.625,5775.0,5300.0,5800.0,383.700000,274.51,429.000000,244.1100,1451.80
1,Dam dispenser #1,AJX75334505,3KPM0016-2,10,550.3,463.8,160.8,130.85,130.85,300.0,...,0.00,5.400,4000.0,9000.0,4000.0,377.300000,282.15,429.283333,244.0500,1451.70
2,Dam dispenser #2,AJX75334501,4E1X9167-1,16,162.4,465.4,551.7,124.00,130.85,300.0,...,0.00,5.400,5800.0,5300.0,5800.0,1309.166667,274.33,1323.616667,243.9575,1475.45
3,Dam dispenser #2,AJX75334501,3K1X0057-1,10,162.4,465.5,552.0,130.85,130.85,300.0,...,0.00,5.400,4000.0,5000.0,4000.0,1271.433333,282.15,1323.516667,243.9000,1476.15
4,Dam dispenser #1,AJX75334501,3HPM0007-1,10,549.0,462.5,159.5,133.50,133.50,300.0,...,0.02,5.400,9000.0,9000.0,9000.0,377.333333,284.80,429.183333,244.0500,1451.70


# 범주형을 수치형으로 변환

In [262]:
# 인코딩할 열 이름의 접미사 리스트
suffixes = ['Dam', 'Fill1', 'Fill2']

# LabelEncoder 객체 생성
le = LabelEncoder()

# 각 접미사에 대해 LabelEncoder를 적용
for suffix in suffixes:
    for column in [f'Equipment_{suffix}']:
        all_data[column] = all_data[column].astype(str)
        le.fit(all_data[column])
        all_data[column] = le.transform(all_data[column])


In [263]:
all_data['Model.Suffix_Dam'] = all_data['Model.Suffix_Dam'].astype(str)
le.fit(all_data['Model.Suffix_Dam'])
all_data['Model.Suffix_Dam'] = le.transform(all_data['Model.Suffix_Dam'])

In [264]:
all_data['Workorder_Dam'] = all_data['Workorder_Dam'].astype(str)
le.fit(all_data['Workorder_Dam'])
all_data['Workorder_Dam'] = le.transform(all_data['Workorder_Dam'])

In [265]:
all_data['Chamber Temp. Judge Value_AutoClave'] = all_data['Chamber Temp. Judge Value_AutoClave'].astype(str)
le.fit(all_data['Chamber Temp. Judge Value_AutoClave'])
all_data['Chamber Temp. Judge Value_AutoClave'] = le.transform(all_data['Chamber Temp. Judge Value_AutoClave'])

# 중복되는 값 있는지 확인

In [267]:
# 두 열의 값이 같은지 비교
equal_rows = all_data['Equipment_Dam'] != all_data['Equipment_Fill2']

# 값이 같은 행들의 개수 구하기
num_equal_rows = equal_rows.sum()

# 결과 확인
print(f"Number of rows where the values are equal: {num_equal_rows}")

# 값이 같은 행들의 인덱스 추출
equal_indices = equal_rows[equal_rows].index

# 결과 확인
print(f"Indices of rows where the values are equal: {equal_indices.tolist()}")

Number of rows where the values are equal: 46
Indices of rows where the values are equal: [3364, 4787, 5102, 5561, 5922, 6560, 6563, 9724, 10007, 13028, 14452, 15612, 17078, 17503, 20754, 20920, 21443, 21841, 22804, 25344, 25525, 25635, 27170, 27472, 28371, 28409, 30187, 31191, 31260, 32124, 32946, 34915, 37520, 39245, 562, 3457, 7287, 7836, 8253, 8898, 10989, 12439, 15180, 15406, 15811, 15964]


In [268]:
# 두 열의 값이 같은지 비교
equal_rows = all_data['PalletID Collect Result_Dam'] != all_data['PalletID Collect Result_Fill2']

# 값이 같은 행들의 개수 구하기
num_equal_rows = equal_rows.sum()

# 결과 확인
print(f"Number of rows where the values are equal: {num_equal_rows}")

# 값이 같은 행들의 인덱스 추출
equal_indices = equal_rows[equal_rows].index

# 결과 확인
print(f"Indices of rows where the values are equal: {equal_indices.tolist()}")

Number of rows where the values are equal: 108
Indices of rows where the values are equal: [137, 1079, 1832, 2118, 2164, 2544, 2724, 2808, 3251, 3364, 4028, 4277, 4787, 5102, 5437, 5561, 5922, 6485, 6531, 6560, 6563, 6860, 7471, 7798, 8066, 8555, 9081, 9724, 10007, 10314, 10410, 12736, 12980, 13009, 13028, 14452, 15612, 17078, 17503, 17926, 18190, 18868, 18941, 20194, 20754, 20920, 21339, 21443, 21812, 21841, 21933, 21967, 22068, 22309, 22804, 24006, 24876, 25344, 25525, 25635, 27170, 27445, 27472, 28349, 28371, 28409, 30187, 31191, 31260, 31640, 32124, 32946, 33477, 34236, 34572, 34915, 35599, 36534, 36582, 37520, 37886, 38377, 39245, 64, 562, 1460, 1530, 1892, 2505, 2710, 3457, 3682, 4932, 7001, 7287, 7666, 7836, 8253, 8898, 10989, 12439, 12585, 12844, 14756, 15180, 15406, 15811, 15964]


In [269]:
# 두 열의 값이 같은지 비교
equal_rows = all_data['Production Qty Collect Result_Dam'] != all_data['Production Qty Collect Result_Fill2']

# 값이 같은 행들의 개수 구하기
num_equal_rows = equal_rows.sum()

# 결과 확인
print(f"Number of rows where the values are equal: {num_equal_rows}")

# 값이 같은 행들의 인덱스 추출
equal_indices = equal_rows[equal_rows].index

# 결과 확인
print(f"Indices of rows where the values are equal: {equal_indices.tolist()}")

Number of rows where the values are equal: 121
Indices of rows where the values are equal: [137, 1079, 1832, 2118, 2164, 2544, 2724, 2766, 2808, 3251, 3364, 4028, 4277, 4787, 5102, 5437, 5561, 5922, 6485, 6531, 6560, 6563, 6860, 7471, 7798, 8066, 8537, 8555, 8878, 9081, 9724, 10007, 10314, 10410, 10914, 12736, 12980, 13009, 13028, 14452, 15612, 17078, 17503, 17693, 17926, 18190, 18868, 18941, 20194, 20221, 20754, 20905, 20920, 21339, 21443, 21812, 21841, 21933, 21967, 22068, 22309, 22804, 24006, 24876, 25344, 25525, 25635, 27170, 27445, 27472, 27523, 28349, 28371, 28409, 30187, 30640, 31191, 31260, 31640, 32124, 32946, 33477, 34236, 34572, 34915, 35599, 36534, 36582, 37520, 37886, 38377, 38762, 39245, 64, 562, 1460, 1530, 1892, 2505, 2710, 3457, 3682, 3732, 4928, 4932, 6092, 7001, 7287, 7666, 7836, 8253, 8898, 10989, 12439, 12585, 12844, 14756, 15180, 15406, 15811, 15964]


In [270]:
l1=['Equipment_Fill1','Equipment_Fill2','PalletID Collect Result_Fill1','PalletID Collect Result_Fill2','Production Qty Collect Result_Fill1','Production Qty Collect Result_Fill2']
all_data = all_data.drop(l1, axis=1)

In [271]:
all_data

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,...,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2,Average Stage1 CL Distance Speed Collect Result_Dam,Average Stage2 CL Distance Speed Collect Result_Dam,Average Stage3 CL Distance Speed Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill
0,0,3,658,16,550.5,464.3,161.4,124.00,130.85,300.0,...,0.000000,5.625000,5775.0,5300.0,5800.0,383.700000,274.510,614.85,429.000000,244.1100
1,0,3,284,10,550.3,463.8,160.8,130.85,130.85,300.0,...,0.000000,5.400000,4000.0,9000.0,4000.0,377.300000,282.150,614.75,429.283333,244.0500
2,1,0,590,16,162.4,465.4,551.7,124.00,130.85,300.0,...,0.000000,5.400000,5800.0,5300.0,5800.0,1309.166667,274.330,904.00,1323.616667,243.9575
3,1,0,252,10,162.4,465.5,552.0,130.85,130.85,300.0,...,0.000000,5.400000,4000.0,5000.0,4000.0,1271.433333,282.150,904.80,1323.516667,243.9000
4,0,0,143,10,549,462.5,159.5,133.50,133.50,300.0,...,0.020000,5.400000,9000.0,9000.0,9000.0,377.333333,284.800,614.75,429.183333,244.0500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,1,0,278,10,162.4,465.5,552.0,130.85,130.85,300.0,...,0.000000,5.400000,4000.0,4000.0,4000.0,1271.433333,282.150,904.80,1323.616667,243.9000
17357,1,0,440,16,162.7,465.7,552.0,130.85,130.85,300.0,...,0.000000,5.400000,6500.0,6250.0,6500.0,1309.166667,280.894,903.95,1323.616667,244.0025
17358,0,0,315,16,551.5,465.0,162.3,130.85,130.85,300.0,...,0.000000,5.400000,6500.0,6500.0,6500.0,377.433333,282.361,614.65,429.383333,244.0500
17359,0,0,8,10,549,463.0,160.5,133.50,133.50,303.5,...,0.196078,5.294118,9000.0,12000.0,9000.0,377.333333,284.800,615.75,429.183333,244.0000


In [272]:
# 'object' 타입 열 선택
object_columns = all_data.select_dtypes(include=['object']).columns

print(f"Object type columns: {object_columns}")

Object type columns: Index(['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam', 'target',
       'Set ID'],
      dtype='object')


In [273]:
all_data.describe(include='all')

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,...,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2,Average Stage1 CL Distance Speed Collect Result_Dam,Average Stage2 CL Distance Speed Collect Result_Dam,Average Stage3 CL Distance Speed Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill
count,57867.000000,57867.000000,57867.000000,57867.000000,57867,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000,...,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000,57867.000000
unique,NaN,NaN,NaN,NaN,38,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,549,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,10422,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.381962,0.348489,308.577116,12.890352,NaN,464.443052,310.354465,129.875025,131.332343,300.414252,...,0.004755,5.444933,6447.031987,7102.976653,6448.482944,726.826750,280.983966,726.989130,770.881099,244.008514
std,0.485872,0.931022,183.916509,2.997935,NaN,1.344287,190.084174,3.294883,1.245883,1.130619,...,0.023837,0.159448,1680.310623,2266.512366,1679.450268,444.336204,3.618679,142.465738,434.478851,0.082390
min,0.000000,0.000000,0.000000,10.000000,NaN,462.200000,159.500000,118.850000,124.500000,300.000000,...,0.000000,3.600000,4000.000000,4000.000000,4000.000000,377.233333,273.800000,614.200000,428.883333,237.833333
25%,0.000000,0.000000,162.000000,10.000000,NaN,463.500000,160.500000,130.850000,130.850000,300.000000,...,0.000000,5.400000,5000.000000,5300.000000,5000.000000,377.333333,281.095000,614.750000,429.283333,243.950000
50%,0.000000,0.000000,299.000000,10.000000,NaN,464.200000,161.500000,130.850000,130.850000,300.000000,...,0.000000,5.400000,6500.000000,6500.000000,6500.000000,377.433333,282.150000,615.750000,429.383333,244.017500
75%,1.000000,0.000000,451.000000,16.000000,NaN,465.400000,551.700000,130.850000,130.850000,300.000000,...,0.000000,5.400000,6500.000000,9000.000000,6500.000000,1271.433333,282.500000,904.300000,1323.616667,244.050000


In [275]:
all_data.info(object)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57867 entries, 0 to 17360
Data columns (total 48 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Equipment_Dam                                             57867 non-null  int32  
 1   Model.Suffix_Dam                                          57867 non-null  int32  
 2   Workorder_Dam                                             57867 non-null  int32  
 3   DISCHARGED SPEED OF RESIN Collect Result_Dam              57867 non-null  int64  
 4   HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam  57867 non-null  object 
 5   HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam  57867 non-null  float64
 6   HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam  57867 non-null  float64
 7   Head Clean Position Z Collect Result_Dam                  57867 non-null  float64
 8   Head Purge Posit

In [276]:
all_data.to_csv('all_data.csv', index=False)

In [277]:
# # 피처 간 상관 관계 매트릭스 ❶
# corrMat = all_data.corr()
# fig, ax= plt.subplots()
# fig.set_size_inches(10, 10)
# sns.heatmap(corrMat, annot=True) # 상관관계 히트맵 그리기 ❷
# ax.set(title='Heatmap of Numerical Data')

# 연습

In [278]:
all_data.head()

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,...,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2,Average Stage1 CL Distance Speed Collect Result_Dam,Average Stage2 CL Distance Speed Collect Result_Dam,Average Stage3 CL Distance Speed Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill
0,0,3,658,16,550.5,464.3,161.4,124.00,130.85,300.0,...,0.00,5.625,5775.0,5300.0,5800.0,383.700000,274.51,614.85,429.000000,244.1100
1,0,3,284,10,550.3,463.8,160.8,130.85,130.85,300.0,...,0.00,5.400,4000.0,9000.0,4000.0,377.300000,282.15,614.75,429.283333,244.0500
2,1,0,590,16,162.4,465.4,551.7,124.00,130.85,300.0,...,0.00,5.400,5800.0,5300.0,5800.0,1309.166667,274.33,904.00,1323.616667,243.9575
3,1,0,252,10,162.4,465.5,552.0,130.85,130.85,300.0,...,0.00,5.400,4000.0,5000.0,4000.0,1271.433333,282.15,904.80,1323.516667,243.9000
4,0,0,143,10,549,462.5,159.5,133.50,133.50,300.0,...,0.02,5.400,9000.0,9000.0,9000.0,377.333333,284.80,614.75,429.183333,244.0500


In [279]:
# 훈련 데이터와 테스트 데이터 나누기 ❶
train_x = all_data[~pd.isnull(all_data['target'])] 
test_x = all_data[pd.isnull(all_data['target'])]

# 타깃값 count 제거 ❷
train_x = train_x.drop(['target'], axis=1)
test_x = test_x.drop(['target'], axis=1)
train_y = train_X['target'] # 타깃값 ❸

In [281]:
train_x.to_csv('t.csv', index=False)

## 연습

In [282]:
train_x = train_x.drop(['Set ID'],axis=1)

train_y = train_y.astype(str)
train_y = le.fit_transform(train_y)

test_x['Set ID'] = test_x['Set ID'].astype(str)
test_x['Set ID'] = le.fit_transform(test_x['Set ID'])

In [283]:
test_x

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,...,CURE DpS Z Collect Result_Fill2,CURE DpS Θ Collect Result_Fill2,Average Stage1 CL Distance Speed Collect Result_Dam,Average Stage2 CL Distance Speed Collect Result_Dam,Average Stage3 CL Distance Speed Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill
0,1,0,241,10,162.4,465.3,551.3,130.85,130.85,300.0,...,0.000000,5.400000,5000.0,9000.0,5000.0,1271.100000,282.150,904.80,1323.516667,243.9000
1,1,0,510,16,162.4,465.4,551.7,124.00,130.85,300.0,...,0.000000,5.400000,6000.0,5500.0,6000.0,1309.166667,274.200,904.45,1323.616667,244.0025
2,0,0,128,10,549.5,463.0,160.0,133.50,133.50,300.0,...,0.200000,5.400000,9000.0,9000.0,9000.0,377.333333,284.600,614.75,429.383333,243.9500
3,1,0,307,10,164.2,467.1,553.6,130.85,130.85,300.0,...,0.000000,5.400000,4000.0,4000.0,4000.0,1271.433333,282.150,904.80,1323.616667,243.9000
4,0,0,416,16,550.4,463.7,161.3,130.85,130.85,300.0,...,0.000000,5.400000,6500.0,6250.0,6500.0,377.433333,281.222,614.70,429.383333,244.0500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,1,0,278,10,162.4,465.5,552.0,130.85,130.85,300.0,...,0.000000,5.400000,4000.0,4000.0,4000.0,1271.433333,282.150,904.80,1323.616667,243.9000
17357,1,0,440,16,162.7,465.7,552.0,130.85,130.85,300.0,...,0.000000,5.400000,6500.0,6250.0,6500.0,1309.166667,280.894,903.95,1323.616667,244.0025
17358,0,0,315,16,551.5,465.0,162.3,130.85,130.85,300.0,...,0.000000,5.400000,6500.0,6500.0,6500.0,377.433333,282.361,614.65,429.383333,244.0500
17359,0,0,8,10,549,463.0,160.5,133.50,133.50,303.5,...,0.196078,5.294118,9000.0,12000.0,9000.0,377.333333,284.800,615.75,429.183333,244.0000


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

# Ensure the same columns in the same order for train and test datasets
common_cols = train_x.columns.intersection(test_x.columns)

train_x_line_0 = train_x[train_x['Equipment_Dam'] == 0][common_cols].drop(['Equipment_Dam'], axis=1)
train_y_line_0 = train_y[train_x['Equipment_Dam'] == 0]

train_x_line_1 = train_x[train_x['Equipment_Dam'] == 1][common_cols].drop(['Equipment_Dam'], axis=1)
train_y_line_1 = train_y[train_x['Equipment_Dam'] == 1]

test_x_line_0 = test_x[test_x['Equipment_Dam'] == 0][common_cols].drop(['Equipment_Dam'], axis=1)
test_x_line_1 = test_x[test_x['Equipment_Dam'] == 1][common_cols].drop(['Equipment_Dam'], axis=1)

# 모델 학습 및 예측 함수
def train_and_predict_with_gridsearch(train_x, train_y, test_x, param_grid, cv_folds=10):
    skf = StratifiedKFold(n_splits=cv_folds, random_state=37, shuffle=True)

    # Random Forest 모델 초기화
    xgb_model = XGBClassifier(
        random_state=37,
        tree_method='gpu_hist',    # GPU 사용 설정
        predictor='gpu_predictor'  # GPU 기반 예측 설정
    )

    # GridSearchCV 초기화
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=skf, n_jobs=-1, verbose=2)


    grid_search.fit(train_x, train_y)

     # 최적 모델로 테스트 데이터 예측
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict_proba(test_x)[:, 1]

    return y_pred

# 하이퍼파라미터 후보 설정
param_grid = {
    'n_estimators': [100, 200, 300],  # 트리의 개수
    'max_depth': [6, 8, 12, 16],      # 트리의 최대 깊이
    'learning_rate': [0.01, 0.1, 0.2],  # 학습률
    'subsample': [0.8, 1.0],          # 샘플링 비율
    'colsample_bytree': [0.8, 1.0]    # 트리 생성 시 컬럼 샘플링 비율
}

# XGBoost
y_pred_line_0_xgb = train_and_predict_with_gridsearch(train_x_line_0, train_y_line_0, test_x_line_0, param_grid)
y_pred_line_1_xgb = train_and_predict_with_gridsearch(train_x_line_1, train_y_line_1, test_x_line_1, param_grid)

# 예측 결과 결합 (두 모델의 예측 결과의 평균을 사용)
final_pred = np.zeros(len(test_x))

final_pred[test_x['Equipment_Dam'] == 0] = y_pred_line_0_xgb  
final_pred[test_x['Equipment_Dam'] == 1] = y_pred_line_1_xgb

# 이진 분류 결과로 변환
final_pred_binary = (final_pred > 0.5).astype(int)


# 결과를 DataFrame으로 저장
result = pd.DataFrame({
    'Set ID': test_x['Set ID'],
    'target': final_pred_binary
})

# 결과 출력
print(result.head())


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# Ensure the same columns in the same order for train and test datasets
common_cols = train_x.columns.intersection(test_x.columns)

train_x_line_0 = train_x[train_x['Equipment_Dam'] == 0][common_cols].drop(['Equipment_Dam'], axis=1)
train_y_line_0 = train_y[train_x['Equipment_Dam'] == 0]

train_x_line_1 = train_x[train_x['Equipment_Dam'] == 1][common_cols].drop(['Equipment_Dam'], axis=1)
train_y_line_1 = train_y[train_x['Equipment_Dam'] == 1]

test_x_line_0 = test_x[test_x['Equipment_Dam'] == 0][common_cols].drop(['Equipment_Dam'], axis=1)
test_x_line_1 = test_x[test_x['Equipment_Dam'] == 1][common_cols].drop(['Equipment_Dam'], axis=1)

# 모델 학습 및 예측 함수
def train_and_predict(train_x, train_y, test_x, model):
    skf = StratifiedKFold(n_splits=10, random_state=37, shuffle=True)
    y_pred = np.zeros(len(test_x))

    for train_index, val_index in skf.split(train_x, train_y):
        X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
        y_train, y_val = train_y[train_index], train_y[val_index]

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

        y_pred += model.predict_proba(test_x)[:, 1] / skf.n_splits
    
    return y_pred

# 모델 초기화
catboost_model = CatBoostClassifier(iterations=1000, random_seed=37, learning_rate=0.1, max_depth=6, task_type="GPU", verbose=100)
xgb_model = XGBClassifier(
    n_estimators=1000,
    random_state=37,
    learning_rate=0.1,
    max_depth=6,
    use_label_encoder=False,
    eval_metric='logloss',
    tree_method='gpu_hist',    # GPU 사용 설정
    predictor='gpu_predictor'  # GPU 기반 예측 설정
)

lgbm_model = LGBMClassifier( n_estimators=1000,
    random_state=37,
    learning_rate=0.1,
    max_depth=10,            # 트리의 최대 깊이 증가
    min_child_samples=5,     # 리프 노드에서 최소한의 데이터 포인트 수 감소
    device="gpu")

# 라인별로 모델 학습 및 예측
# CatBoost
y_pred_line_0_cat = train_and_predict(train_x_line_0, train_y_line_0, test_x_line_0, catboost_model)
y_pred_line_1_cat = train_and_predict(train_x_line_1, train_y_line_1, test_x_line_1, catboost_model)

# XGBoost
y_pred_line_0_xgb = train_and_predict(train_x_line_0, train_y_line_0, test_x_line_0, xgb_model)
y_pred_line_1_xgb = train_and_predict(train_x_line_1, train_y_line_1, test_x_line_1, xgb_model)

# LightGBM
y_pred_line_0_lgbm = train_and_predict(train_x_line_0, train_y_line_0, test_x_line_0, lgbm_model)
y_pred_line_1_lgbm = train_and_predict(train_x_line_1, train_y_line_1, test_x_line_1, lgbm_model)

# 예측 결과 결합 (두 모델의 예측 결과의 평균을 사용)
final_pred = np.zeros(len(test_x))

final_pred[test_x['Equipment_Dam'] == 0] = (
    (y_pred_line_0_cat + y_pred_line_0_xgb + y_pred_line_0_lgbm) / 3
)
final_pred[test_x['Equipment_Dam'] == 1] = (
    (y_pred_line_1_cat + y_pred_line_1_xgb + y_pred_line_1_lgbm) / 3
)

# 이진 분류 결과로 변환
final_pred = (final_pred > 0.5).astype(int)

# 결과를 DataFrame으로 저장
result = pd.DataFrame({
    'Set ID': test_x['Set ID'],
    'target': final_pred
})

# 결과 출력
print(result.head())


In [ ]:
# 기존 0과 1 값을 'AbNormal'과 'Normal'로 변환
submission['target'] = result['target'].map({0: 'AbNormal', 1: 'Normal'})


# 결과를 CSV 파일로 저장
submission.to_csv('submission.csv', index=False)

# 결과 확인
submission.head()


,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal


In [ ]:
unique, counts = np.unique(submission['target'], return_counts=True)
print(unique, counts)

['AbNormal' 'Normal'] [   77 17284]


In [ ]:
train_x.head()

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,...,DISCHARGED VpS RESIN(Stage3) Collect Result_Fill1,Average Stage1 CL Distance Speed Collect Result_Dam,Average Stage2 CL Distance Speed Collect Result_Dam,Average Stage3 CL Distance Speed Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage) Collect Result_Fill2
0,0,3,658,16,550.5,464.3,161.4,124.00,130.85,300.0,...,0.95000,5775.0,5300.0,5800.0,383.700000,274.51,430.033333,244.520,427.966667,243.7
1,0,3,284,10,550.3,463.8,160.8,130.85,130.85,300.0,...,0.94963,4000.0,9000.0,4000.0,377.300000,282.15,430.600000,244.400,427.966667,243.7
2,1,0,590,16,162.4,465.4,551.7,124.00,130.85,300.0,...,0.95000,5800.0,5300.0,5800.0,1309.166667,274.33,1323.033333,244.415,1324.200000,243.5
3,1,0,252,10,162.4,465.5,552.0,130.85,130.85,300.0,...,0.94963,4000.0,5000.0,4000.0,1271.433333,282.15,1322.833333,244.300,1324.200000,243.5
4,0,0,143,10,549.0,462.5,159.5,133.50,133.50,300.0,...,0.94963,9000.0,9000.0,9000.0,377.333333,284.80,430.400000,244.400,427.966667,243.7


In [ ]:
train_x.to_csv('train_x.csv', index=False)